In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!git clone https://github.com/dalibra/MMD.git

Cloning into 'MMD'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 39 (delta 3), reused 0 (delta 0), pack-reused 30
Receiving objects: 100% (39/39), 15.40 MiB | 20.69 MiB/s, done.
Resolving deltas: 100% (16/16), done.


In [3]:
cd MMD

/kaggle/working/MMD


In [4]:
!pip install wandb -q

In [5]:
# !pip install theano -q

In [6]:
# !sed -i 's/blas_info = np.distutils.__config__.blas_opt_info/blas_info = np.distutils.__config__.blas_ilp64_opt_info/' "/usr/local/lib/python3.10/dist-packages/theano/configdefaults.py"
# !sed -n '1284p' "/usr/local/lib/python3.10/dist-packages/theano/configdefaults.py"

In [7]:
# !python Example_VAE.py -s mnist.npy

# Libraries

In [8]:
import json
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import wandb
!wandb login 1b8e8dc9dcf1a34397a04197c4826d3fe7441dae

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
cuda


In [9]:
# import VariationalAutoencoder
import numpy as np
import scipy as sp
import time,os
import _pickle as cPickle
import gzip,copy,pickle
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from mmd import MMD_3_Sample_Test

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Data loading

In [10]:
print("Loading MNIST data")
#Retrieved from: http://deeplearning.net/data/mnist/mnist.pkl.gz

f = gzip.open('mnist.pkl.gz', 'rb')
(x_train, t_train), (x_valid, t_valid), (x_test, t_test)  = cPickle.load(f, encoding='latin1')
f.close()
x_train=(x_train>0).astype('float')
x_valid=(x_valid>0).astype('float')
x_test=(x_test>0).astype('float')

print(x_train.shape, t_train.shape, x_valid.shape, t_valid.shape, x_test.shape, t_test.shape)

Loading MNIST data
(50000, 784) (50000,) (10000, 784) (10000,) (10000, 784) (10000,)


# VAE

In [11]:
class encoder(torch.nn.Module):
    def __init__(self, args):
        super().__init__()

        self.input_dim = args["input_dim"]
        self.hidden_dim = args["hidden_dim"]
        self.latent_dim = args["latent_dim"]*2

        self.layers = nn.Sequential(
            nn.Linear(self.input_dim, self.hidden_dim),
            nn.Sigmoid(),
            nn.Linear(self.hidden_dim, self.latent_dim)
        )

    def forward(self, x: torch.Tensor):
        return self.layers(x)

class decoder(torch.nn.Module):
    def __init__(self, args):
        super().__init__()

        self.latent_dim = args["latent_dim"]
        self.hidden_dim = args["hidden_dim"]
        self.input_dim = args["input_dim"]

        self.layers = nn.Sequential(
            nn.Linear(self.latent_dim, self.hidden_dim),
            nn.Sigmoid(),
            nn.Linear(self.hidden_dim, self.input_dim)
        )

    def forward(self, x: torch.Tensor):
        return self.layers(x)

#code from https://github.com/1Konny/Beta-VAE/blob/master

def reconstruction_loss(x, x_recon, distribution):
    batch_size = x.size(0)
    assert batch_size != 0

    if distribution == 'bernoulli':
        recon_loss = F.binary_cross_entropy_with_logits(x_recon, x, size_average=False).div(batch_size)
    elif distribution == 'gaussian':
#         x_recon = F.sigmoid(x_recon)
        recon_loss = F.mse_loss(x_recon, x, size_average=False).div(batch_size)
    else:
        raise ValueError("Unknown distribution")

    return recon_loss


def kl_divergence(mu, logvar):
    batch_size = mu.size(0)
    assert batch_size != 0
    if mu.data.ndimension() == 4:
        mu = mu.view(mu.size(0), mu.size(1))
    if logvar.data.ndimension() == 4:
        logvar = logvar.view(logvar.size(0), logvar.size(1))

    klds = -0.5*(1 + logvar - mu.pow(2) - logvar.exp())
    total_kld = klds.sum(1).mean(0, True)
    dimension_wise_kld = klds.mean(0)
    mean_kld = klds.mean(1).mean(0, True)

    return total_kld, dimension_wise_kld, mean_kld


def reparametrize(mu, logvar):
    std = logvar.div(2).exp()
    eps = Variable(std.data.new(std.size()).normal_())
    return mu + std*eps


class View(nn.Module):
    def __init__(self, size):
        super(View, self).__init__()
        self.size = size

    def forward(self, tensor):
        return tensor.view(self.size)


class BetaVAE_H(nn.Module):
    """Model proposed in original beta-VAE paper(Higgins et al, ICLR, 2017)."""

    def __init__(self, encoder, decoder, **args):
        super(BetaVAE_H, self).__init__()
        
        self.input_dim = args["input_dim"]
        self.z_dim = args["latent_dim"]
        self.beta = args["beta"]
        self.decoder_dist = args["data_prior"] #"gaussian"/"bernoulli"
        self.device = args["device"]
        
        self.encoder = encoder
        self.decoder = decoder

        self.prior_mu = torch.zeros(self.z_dim).to(device)
        self.log_prior_sigma = torch.zeros(self.z_dim).to(device)
        self.rng = np.random.default_rng()

    def forward(self, x):
        x_recon, mu, logvar = self._reconstruct(x)
        
        recon_loss = reconstruction_loss(x, x_recon, self.decoder_dist)
        total_kld, dim_wise_kld, mean_kld = kl_divergence(mu, logvar)
        
        beta_vae_loss = recon_loss + self.beta*total_kld

        return {
            'loss': beta_vae_loss,
            '-log p(x|z)': recon_loss,
            'kl': total_kld,
            'decoded_imgs': x_recon
        }

    def _encode(self, x):
        return self.encoder(x)
    
    def encode(self, x):
        distributions = self._encode(x)
        return distributions[:, :self.z_dim]

    def _decode(self, z):
        return self.decoder(z)
    
    def _reconstruct(self, x):
        distributions = self._encode(x)
        mu = distributions[:, :self.z_dim]
        logvar = distributions[:, self.z_dim:]
        z = reparametrize(mu, logvar)
        x_recon = self._decode(z)
        return x_recon, mu, logvar
    
    def reconstruct(self, x):
        x_recon, mu, logvar = self._reconstruct(x)
        return x_recon
    
    def sample_from_p_z(self, num_samples):
        return self.prior_mu + torch.exp(self.log_prior_sigma)*torch.tensor(
            self.rng.multivariate_normal(mean=np.ones(self.z_dim), cov=np.eye(self.z_dim), size=num_samples), 
            requires_grad=False
        ).to(torch.float32).to(self.device)
    
    def sample(self, num_samples):
        return self._decode(self.sample_from_p_z(num_samples))
    
class Trainer():
    def __init__(self, model, lr, bs, device):
        self.lr = lr
        self.bs = bs
        self.device = device
        self.model = self.to_device(model)
        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.lr)
        
    def to_cpu(self, x):
        return x.to("cpu")
        
    def to_device(self, x):
        return x.to(self.device)
        
    def create_dataloader(self, data):
        return DataLoader(data, shuffle=True, batch_size = self.bs, num_workers=0)
        
    def iterate(self, data):
        self.model.train()
        dataloader = self.create_dataloader(data)
        for batch in dataloader:
            batch = self.to_device(batch).to(torch.float32)
            loss = self.model(batch)["loss"]
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            
    def getLowerBound(self, data):
        self.model.eval()
        dataloader = self.create_dataloader(data)
        running_loss = 0
        for batch in dataloader:
            batch = self.to_device(batch).to(torch.float32)
            loss = self.model(batch)["loss"]
            running_loss += loss
        return -running_loss/len(dataloader)
    
    def sample(self, N):
        return self.to_cpu(self.model.sample(N)).detach().numpy()
    
    def encode(self, x):
        return self.model.encode(self.to_device(torch.tensor(x).to(torch.float32))).detach().cpu().numpy()

# Train config

In [25]:
data_train=x_train
samp_size=5000
# samp_size=1000
verbose=True
runs=10
# runs=1
# ratios=np.array([0.9, 4.0, 0.5, 2.0, 0.8, 3.0, 1.5, 1.9, 2.5])
# ratios=np.array([1.0])
# t_size2=1500
t_size = 25000
# t_size = 1000

dimZ = 20
HU_decoder = 400
HU_encoder = HU_decoder
# dimZ2 = 20
# HU_decoder2 = 400
# HU_encoder2 = HU_decoder2
dimZ2_HU_decoder2_list = [(50, 400)]
# dimZ2_HU_decoder2_list = [(5, 200)]

batch_size = 100
L = 1
learning_rate = 0.01
sig=0.05
# maxiter=10
maxiter=2000

In [26]:
for dimZ2, HU_decoder2 in tqdm(dimZ2_HU_decoder2_list):
    HU_encoder2=HU_decoder2
#     t_size1=int(t_size2/ratio)
#     print(f"{t_size1 = }, {t_size2 = }")

    config={
        "samp_size": samp_size,
        "runs": runs,
#         "ratio": ratio,
#         "t_size2": t_size2,
#         "t_size1": t_size1,
        "t_size": t_size,
        "dimZ": dimZ,
        "HU_decoder": HU_decoder,
        "HU_encoder": HU_encoder,
        "dimZ2": dimZ2,
        "HU_decoder2": HU_decoder2,
        "HU_encoder2": HU_encoder2,
        "batch_size": batch_size,
        "L": L,
        "learning_rate": learning_rate,
        "sig": sig,
        "maxiter": maxiter
    }
    print(config)

    results = {
        "pvalue_win1" : [],
        "tstat_win1": [],
        "sigma_win1": [],
        "pvalue_win2" : [],
        "tstat_win2": [],
        "sigma_win2": [],
        "n_wins_1": 0,
        "n_wins_2": 0,
        "n_draws": 0
    }

    #dataset
#     set1,set2 = train_test_split(range(data.shape[0]), train_size=t_size1+t_size2)
    set1,set2 = train_test_split(range(data_train.shape[0]), train_size=t_size)
#     data1 = data[set1[0:t_size1],:]
#     data2 = data[set1[t_size1:t_size2+t_size1],:]
    data = data_train[set1[0:t_size],:]
    set2=set2[0:samp_size]
    data_holdout=data_train[set2,:]
#     print(f"{len(data1) = }, {len(data2) = }, {len(data_holdout) = }")
    print(f"{len(data) = }, {len(data_holdout) = }")
#     [N1,dimX] = data1.shape
#     [N2,dimX] = data2.shape
    [N,dimX] = data.shape

    #model initialization    
#     encoder1 = VariationalAutoencoder.VA(HU_decoder, HU_encoder, dimX, dimZ, batch_size, L, learning_rate, continous=False)
#     encoder2 = VariationalAutoencoder.VA(HU_decoder2, HU_encoder2, dimX, dimZ2, batch_size, L, learning_rate, continous=False)
#     encoder1.createGradientFunctions()
#     encoder2.createGradientFunctions()
#     encoder1.initParams()
#     encoder2.initParams()
    model_config = {
        "input_dim" : dimX,
        "hidden_dim": HU_decoder,
        "latent_dim": dimZ,
        "data_prior": "bernoulli",
        "beta": 1,
        "device": device
    }
    model_config2 = {
        "input_dim" : dimX,
        "hidden_dim": HU_decoder2,
        "latent_dim": dimZ2,
        "data_prior": "bernoulli",
        "beta": 1,
        "device": device
    }
    model1 = BetaVAE_H(encoder(model_config), decoder(model_config), **model_config)
    model2 = BetaVAE_H(encoder(model_config2), decoder(model_config2), **model_config2)
    encoder1 = Trainer(model1, learning_rate, batch_size, device)
    encoder2 = Trainer(model2, learning_rate, batch_size, device)

    begin = time.time()
    testlowerbound1=testlowerbound2=-np.Inf

    #training
    run = wandb.init(name=f"hu2={HU_decoder2}, dimz2={dimZ2}", project="Maximum mean discrepancy test", config=config)

    wandb.define_metric("vae1_step")
    wandb.define_metric("lb_train_1", step_metric="vae1_step")
    wandb.define_metric("lb_test_1", step_metric="vae1_step")

    for j in tqdm(range(maxiter)):
#         encoder1.iterate(data1)
        encoder1.iterate(data)
        if j%1 == 0:
            oldlower=testlowerbound1
#             train_lower1=encoder1.getLowerBound(data1).detach()
            train_lower1=encoder1.getLowerBound(data).detach()
            testlowerbound1 = encoder1.getLowerBound(data_holdout).detach()
#             lbtrain = train_lower1/float(N1)
#             lbtest = testlowerbound1/samp_size
            lbtrain = train_lower1
            lbtest = testlowerbound1

            if(verbose):
                print(f"Encoder 1 Iteration {j}| lower bound train = {lbtrain} |lower bound test 1 = {lbtest}")
            wandb.log({"vae1_step": j, "lb_train_1": lbtrain, "lb_test_1": lbtest})

            if(oldlower>=testlowerbound1): break
            best_encoder1=copy.deepcopy(encoder1)

    encoder1=best_encoder1

    wandb.define_metric("vae2_step")
    wandb.define_metric("lb_train_2", step_metric="vae2_step")
    wandb.define_metric("lb_test_2", step_metric="vae2_step")

    for j in tqdm(range(maxiter)):
#         encoder2.iterate(data2)
        encoder2.iterate(data)

        if j%1 == 0:
            oldlower=testlowerbound2
#             train_lower2=encoder2.getLowerBound(data2).detach()
            train_lower2=encoder2.getLowerBound(data).detach()
            testlowerbound2 = encoder2.getLowerBound(data_holdout).detach()
#             lbtrain = train_lower2/float(N2)
#             lbtest = testlowerbound2/samp_size
            lbtrain = train_lower2
            lbtest = testlowerbound2

            if(verbose):
                print("Encoder 2 Iteration %d| lower bound train = %.2f |lower bound test 1= %.2f"
                    % (j, lbtrain, lbtest))
            wandb.log({"vae2_step": j, "lb_train_2": lbtrain, "lb_test_2": lbtest})

            if(oldlower>testlowerbound2):
                break
            best_encoder2=copy.deepcopy(encoder2)

    encoder2=best_encoder2

    end=time.time()

    #test
    for _ in tqdm(range(runs)):
        samples1=encoder1.sample(N=samp_size)
        samples2=encoder2.sample(N=samp_size)

        pvalue_win1, tstat_win1, sigma_win1, MMDXY_win1, MMDXZ_win1 = MMD_3_Sample_Test(data_holdout, samples1, samples2, computeMMDs=False)
        print("win1: pvalue: %.2f"%(pvalue_win1))
        results["pvalue_win1"].append(pvalue_win1)
        results["tstat_win1"].append(tstat_win1)
        results["sigma_win1"].append(sigma_win1)
        pvalue_win2, tstat_win2, sigma_win2, MMDXY_win2, MMDXZ_win2 = MMD_3_Sample_Test(data_holdout, samples2, samples1, computeMMDs=False)
        print("win2: pvalue: %.2f"%(pvalue_win2))
        results["pvalue_win2"].append(pvalue_win2)
        results["tstat_win2"].append(tstat_win2)
        results["sigma_win2"].append(sigma_win2)

        alpha = 0.05
        if pvalue_win1 <= alpha and pvalue_win2 > alpha:
            results["n_wins_2"] += 1
        elif pvalue_win1 > alpha and pvalue_win2 <= alpha:
            results["n_wins_1"] += 1
        else:
            results["n_draws"] += 1
    print("n_wins_1:", results["n_wins_1"], ", n_wins_2:", results["n_wins_2"], ", n_draws:", results["n_draws"])

    data1_enc=encoder1.encode(x_valid)
    data2_enc=encoder2.encode(x_valid)
    test1_enc=encoder1.encode(x_test)
    test2_enc=encoder2.encode(x_test)

    LogReg_VAE = linear_model.LogisticRegression(max_iter=1000)
    LogReg_VAE.fit(data1_enc,t_valid)
    vae1_score = LogReg_VAE.score(test1_enc, t_test)
    LogReg_VAE = linear_model.LogisticRegression(max_iter=1000)
    LogReg_VAE.fit(data2_enc,t_valid)
    vae2_score = LogReg_VAE.score(test2_enc, t_test)
    print("Accuracy 1:%.2f Accuracy2:%.2f"%(vae1_score,vae2_score))

#     wandb.define_metric("ratio")
#     wandb.define_metric("n_wins_1", step_metric="ratio")
#     wandb.define_metric("n_wins_2", step_metric="ratio")
#     wandb.define_metric("n_draws", step_metric="ratio")
#     wandb.define_metric("accuracy_1", step_metric="ratio")
#     wandb.define_metric("accuracy_2", step_metric="ratio")
    wandb.log({
#         "ratio": ratio,
        "n_wins_1": results["n_wins_1"],
        "n_wins_2": results["n_wins_2"],
        "n_draws": results["n_draws"],
        "accuracy_1": vae1_score,
        "accuracy_2": vae2_score
    })

    wandb.finish()

    with open(f"log_hu2={HU_decoder2}_dimz2={dimZ2}.json", "w") as f:
        json.dump(results, f, indent=4)

  0%|          | 0/1 [00:00<?, ?it/s]

{'samp_size': 5000, 'runs': 10, 't_size': 25000, 'dimZ': 20, 'HU_decoder': 400, 'HU_encoder': 400, 'dimZ2': 50, 'HU_decoder2': 400, 'HU_encoder2': 400, 'batch_size': 100, 'L': 1, 'learning_rate': 0.01, 'sig': 0.05, 'maxiter': 2000}
len(data) = 25000, len(data_holdout) = 5000


  0%|          | 0/2000 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Encoder 1 Iteration 0| lower bound train = tensor([-160.0698], device='cuda:0') |lower bound test 1 = tensor([-159.8709], device='cuda:0')
Encoder 1 Iteration 1| lower bound train = tensor([-127.0230], device='cuda:0') |lower bound test 1 = tensor([-127.8976], device='cuda:0')
Encoder 1 Iteration 2| lower bound train = tensor([-113.1850], device='cuda:0') |lower bound test 1 = tensor([-114.2562], device='cuda:0')
Encoder 1 Iteration 3| lower bound train = tensor([-107.4794], device='cuda:0') |lower bound test 1 = tensor([-108.8017], device='cuda:0')
Encoder 1 Iteration 4| lower bound train = tensor([-102.7829], device='cuda:0') |lower bound test 1 = tensor([-104.6312], device='cuda:0')
Encoder 1 Iteration 5| lower bound train = tensor([-100.8791], device='cuda:0') |lower bound test 1 = tensor([-103.0114], device='cuda:0')
Encoder 1 Iteration 6| lower bound train = tensor([-99.5503], device='cuda:0') |lower bound test 1 = tensor([-102.1449], device='cuda:0')
Encoder 1 Iteration 7| lower

...

Encoder 2 Iteration 0| lower bound train = -179.49 |lower bound test 1= -178.78
Encoder 2 Iteration 1| lower bound train = -134.28 |lower bound test 1= -134.65
Encoder 2 Iteration 2| lower bound train = -118.54 |lower bound test 1= -119.45
Encoder 2 Iteration 3| lower bound train = -109.98 |lower bound test 1= -111.62
Encoder 2 Iteration 4| lower bound train = -104.97 |lower bound test 1= -106.76
Encoder 2 Iteration 5| lower bound train = -102.65 |lower bound test 1= -104.79
Encoder 2 Iteration 6| lower bound train = -100.38 |lower bound test 1= -102.77
Encoder 2 Iteration 7| lower bound train = -98.90 |lower bound test 1= -101.71
Encoder 2 Iteration 8| lower bound train = -97.54 |lower bound test 1= -100.31
Encoder 2 Iteration 9| lower bound train = -96.68 |lower bound test 1= -99.82
Encoder 2 Iteration 10| lower bound train = -95.71 |lower bound test 1= -98.84
Encoder 2 Iteration 11| lower bound train = -95.28 |lower bound test 1= -98.80
Encoder 2 Iteration 12| lower bound train = -9

  0%|          | 0/10 [00:00<?, ?it/s]

win1: pvalue: 1.00
win2: pvalue: 0.00
win1: pvalue: 1.00
win2: pvalue: 0.00
win1: pvalue: 1.00
win2: pvalue: 0.00
win1: pvalue: 1.00
win2: pvalue: 0.00
win1: pvalue: 1.00
win2: pvalue: 0.00
win1: pvalue: 1.00
win2: pvalue: 0.00
win1: pvalue: 1.00
win2: pvalue: 0.00
win1: pvalue: 1.00
win2: pvalue: 0.00
win1: pvalue: 1.00
win2: pvalue: 0.00
win1: pvalue: 1.00
win2: pvalue: 0.00
n_wins_1: 10 , n_wins_2: 0 , n_draws: 0
Accuracy 1:0.89 Accuracy2:0.90


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


accuracy_1,▁
accuracy_2,▁
lb_test_1,▁▅▆▇▇▇█████
lb_test_2,▁▅▆▇▇▇▇▇██████████
lb_train_1,▁▅▆▇▇▇▇████
lb_train_2,▁▅▆▇▇▇▇▇██████████
n_draws,▁
n_wins_1,▁
n_wins_2,▁
vae1_step,▁▂▂▃▄▅▅▆▇▇█
vae2_step,▁▁▂▂▃▃▃▄▄▅▅▆▆▆▇▇██


# Link to WandB
https://wandb.ai/dmitriykornilov_team/Maximum%20mean%20discrepancy%20test/workspace?workspace=user-dmitriykornilov